In [36]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tag import CRFTagger
import pycrfsuite
from nltk import word_tokenize,sent_tokenize
import pandas as pd
import numpy as np
import spacy

In [2]:
ct = CRFTagger()
nlp = spacy.load('en')

In [3]:
# Read in the train set data
train_set  = pd.read_csv('Train_and_test_data/train.txt',sep=' ',names=['word','Brill','tag']).drop('Brill',1)
# Format train set data to tuples of a list of lists
train_set = [[tuple(x) for x in train_set.values]]

In [4]:
#Training Code, Keep it commented unless retraining
#ct.train(train_set,'model.crf.tagger') 

sentence = "Once upon a time there was a little girl called Alice . One day Alice met a Rabbit . The Rabbit said to Alice , follow me Alice I will carry you to my Den "


In [5]:
def process_text(passage):
    """
    1.Remove Punctuations and Special characters appear in book chapters
    2.Remove Stopwords
    3.Return Clean list of words
    """
    exclude_set = set(['“','”',':'])

    no_punctuation = [char for char in passage if char not in string.punctuation + "“”."]
    no_punctuation = ''.join(no_punctuation)
    no_punctuation = [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]
    
    return no_punctuation

In [6]:
#remove stop words and punctuation
sentence = process_text(sentence)
sentence = ' '.join(sentence)
print(sentence)

upon time little girl called Alice One day Alice met Rabbit Rabbit said Alice follow Alice carry Den


In [7]:
sentences = sent_tokenize(sentence)
word_list = [[]]
word_list.clear()
wlist = []
i = 0
j = 0

for sent in sentences:
    for word in sent.split():
        wlist.append(word)
        j += 1
        
    wlist.copy
    word_list.append(wlist.copy())
    wlist.clear()
    i += 1
    
print(word_list)

[['upon', 'time', 'little', 'girl', 'called', 'Alice', 'One', 'day', 'Alice', 'met', 'Rabbit', 'Rabbit', 'said', 'Alice', 'follow', 'Alice', 'carry', 'Den']]


In [8]:
#Set the model of previously trained data set
ct.set_model_file(model_file='model.crf.tagger')
word_list = ct.tag_sents(word_list)

#test_set = pd.read_csv('Train_and_test_data/test.txt', sep=' ', names=['words','Brill','tag'], index_col=False).drop(['Brill','tag'],1)
#test_set = test_set.values.tolist()
#ct.tag_sents(test_set)


In [9]:
#test_setEval = pd.read_csv('Train_and_test_data/test.txt', sep=' ', names=['words','Brill','tag']).drop('Brill',1)
#test_setEval = [[tuple(x) for x in test_setEval.values]]
#test the accuracy of the POS
#ct.evaluate(test_setEval)

In [10]:
#Convert list of tuples to a numpy array
word_list = np.array(word_list)

word_list = np.reshape(word_list, (-1,2))

word_list

array([['upon', 'B-PP'],
       ['time', 'B-NP'],
       ['little', 'I-NP'],
       ['girl', 'I-NP'],
       ['called', 'B-VP'],
       ['Alice', 'B-NP'],
       ['One', 'I-NP'],
       ['day', 'I-NP'],
       ['Alice', 'I-NP'],
       ['met', 'I-NP'],
       ['Rabbit', 'I-NP'],
       ['Rabbit', 'I-NP'],
       ['said', 'B-VP'],
       ['Alice', 'B-NP'],
       ['follow', 'I-NP'],
       ['Alice', 'I-NP'],
       ['carry', 'I-NP'],
       ['Den', 'I-NP']], 
      dtype='<U6')

In [11]:
# Find S -> NP VP 
# Returns if a subject is found
# pattern variable = [pat0, pat1, pat2]
def findSubject(pattern):
    #print('{} -> {} {}'.format(pattern[0], pattern[1], pattern[2]))
    if pattern[0] == 'NP' and pattern[1] == 'VP' and pattern[2] == 'NP':
        #if true pattern[0] is the subject of a sentence
        return True
    else:
        return False

In [12]:
#Build Relationship models for the Subject data retrived


In [13]:
#Extract the S -> NP VP to find the subjects in a sentence
#using Markov Chain Model

pattern = []
subjects = []
word_list_length = len(word_list)

for i in range(0, len(word_list)):
    #print(word_list[i][1].split('-')[1])
    try:
        if((word_list_length - i == 2)):
            break
        else:
            pattern = [word_list[i][1].split('-')[1], word_list[i+1][1].split('-')[1], word_list[i+2][1].split('-')[1]]
            #print("index 1: {}, index 2: {}, index 3: {}".format(i, i+1, i+2))
            # Call the function findSubject to identify potential subject elements and save them in an array
            if findSubject(pattern):
                # If returns true consider 1st element as a potential Subject
                print('Potential Subject Found at index {}'.format(i))
                print('Subject -> {}'.format(word_list[i][0]))
                subjects.append(word_list[i][0])
                print('--------------------')
            
    except IndexError:
        # Break the loop if IndexError occurs
        # Fail safe
        print("Out of Index")
        break;

Potential Subject Found at index 3
Subject -> girl
--------------------
Potential Subject Found at index 11
Subject -> Rabbit
--------------------
